### Function attributes

what happens when functions are attributes

So far, we have been dealing with non-callable attributes. When attributes are actually functions, things behave differently.


In [1]:
class Person:
    def say_hello():
        print('Hello!')

In [2]:
Person.say_hello

<function __main__.Person.say_hello()>

In [4]:
type(Person.say_hello)

function

As we can see it is just a plain function, and be called as usual:

In [5]:
Person.say_hello()


Hello!


Now let's create an instance of that class

In [6]:
p = Person()

In [8]:
hex(id(p))

'0x162525cee10'

We know we can access class attributes via the instance, so we should also be able to access the function attribute in the same way

In [9]:
p.say_hello


<bound method Person.say_hello of <__main__.Person object at 0x00000162525CEE10>>

In [10]:
type(p.say_hello)


method


Hmm, the type has changed from function to method, and the function representation states that it is a <b>bound method</b> of the specific object p we created (notice the memory address).

And if we try to call the function from the instance, here's what happens:

In [12]:
p.say_hello()

TypeError: say_hello() takes 0 positional arguments but 1 was given

In [14]:
try:
    p.say_hello()
except Exception as ex:
    print(type(ex).__name__, ex)

TypeError say_hello() takes 0 positional arguments but 1 was given


<b>method</b> is an actual type in Python, and, like functions, they are callables, but they have one distinguishing feature. They need to be bound to an object, and that object reference is passed to the underlying function.

Often when we define functions in a class and call them from the instance we need to know which specific instance was used to call the function. This allows us to interact with the instance variables.

To do this, Python will automatically transform an ordinary function defined in a class into a method when it is called from an instance of the class.

Further, it will "bind" the method to the instance - meaning that the instance will be passed as the first argument to the function being called.

It does this using descriptors.

For now let's just explore this a bit more:

#####  `p.say_hello()`:

* say_hello is a method object.
* it is bound to p (instamce of Program class)
* when p.say_hello is called, the bound object `p` is injected as 1st parameter to the method say_hello.
  so essentially it is calling this: ```Myclass.say_hello(p)```.

So <b> Methods</b> are objects that combines:
1. instance (of some class)
2. function

-> Like any Object it has attributes:
1. `__self__` : the instance the method is bound to
2. `__func__` : the original function (defined in the class)

Calling <b>`obj.method(args)` --> `method.__func__(method.__self__,args)`</b>

#### Instance method: 

this means that we have to account for that "extra" argument(first argument of the method) when we define our function in our classes- otherwise we cannot use them as methods bound to our instances. 

In [15]:
class Person:
    def say_hello(*args):
        print('say_hello args:', args)

In [16]:

Person.say_hello()

say_hello args: ()


As we can see, calling say_hello from the class, just calls the function (it is just a function).

But when we call it from an instance:

In [17]:

p = Person()
hex(id(p))


'0x162529a32e8'

In [19]:
p.say_hello()

say_hello args: (<__main__.Person object at 0x00000162529A32E8>,)


You can see that the object p was passed as an argument to the class function say_hello.

The obvious advantage is that we can now interact with instance attributes easily:

In [20]:

class Person:
    def set_name(instance_obj, new_name):
        instance_obj.name = new_name  # or setattr(instance_obj, 'name', new_name)

In [21]:
p = Person()

In [22]:
p.set_name('Alex')

In [23]:
p.__dict__


{'name': 'Alex'}

This has essentially the same effect as doing this:

In [25]:
Person.set_name(p, 'John')


In [26]:
p.__dict__


{'name': 'John'}

By convention, the first argument is usually named self, but asd you just saw we can name it whatever we want - it just will be in the instance when the method variant of the function is called - and it is called an instance method.

But methods are objects created by Python when calling class functions from an instance.

lets see another example:

In [27]:
class Myclass:
    language = 'python'
    def hello(obj, name):
        return f'Hello {name}! I am {obj.language}'

In [28]:
python = Myclass()

In [29]:
python.hello('Amit')  #Similar to Myclass.hello(python,'Amit')

'Hello Amit! I am python'

In [30]:
Myclass.hello(python,'Amit')

'Hello Amit! I am python'

They have their own unique attributes too:

In [31]:
class Person:
    def say_hello(self):
        print(f'{self} says hello')

In [32]:
p = Person()

In [33]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x00000162529AF860>>

In [34]:
m_hello = p.say_hello

In [35]:
type(m_hello)

method

For example it has a `__func__` attribute:

In [37]:
m_hello.__func__

<function __main__.Person.say_hello(self)>

which happens to be the class function used to create the method (the underlying function).

But remember that a method is bound to an instance. In this case we got the method from the p object:

In [38]:
hex(id(p))

'0x162529af860'

In [39]:
m_hello.__self__

s you can see, the method also has a reference to the object it is bound to.

So think of methods as functions that have been bound to a specific object, and that object is passed in as the first argument of the function call. The remaining arguments are then passed after that.

Instance methods are created automatically for us, when we define functions inside our class definitions.

This even holds true if we monkey-patch our classes at run-time:

In [41]:
class Person:
    def say_hello(self):
        print(f'instance method called from {self}')

In [42]:
p = Person()
hex(id(p))

'0x162529c1438'

In [43]:
p.say_hello()

instance method called from <__main__.Person object at 0x00000162529C1438>


In [44]:
Person.do_work = lambda self: f"do_work called from {self}"


In [45]:
Person.__dict__


mappingproxy({'__module__': '__main__',
              'say_hello': <function __main__.Person.say_hello(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'do_work': <function __main__.<lambda>(self)>})

OK, so both functions are in the class `__dict__`.

let's create an instance and see what happens:

In [46]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x00000162529C1438>>

In [47]:
p.do_work

<bound method <lambda> of <__main__.Person object at 0x00000162529C1438>>

In [48]:
p.do_work()

'do_work called from <__main__.Person object at 0x00000162529C1438>'

But be careful, if we add a function to the <b>instance</b> directly, this does not work the same - we have create a function in the instance, so it is not considered a method (since it was not defined in the class):

In [49]:
p.other_func = lambda *args: print(f'other_func called with {args}')

In [50]:
p.other_func

<function __main__.<lambda>(*args)>

In [51]:
'other_func' in Person.__dict__

False

In [52]:
p.other_func()

other_func called with ()



As you can see, other_func is, and behaves, like an ordinary function.

Long story short, functions defined in a class are transformed into methods when called from instances of the class. So of course, we have to account for that extra argument that is passed to the method.